In [1]:
import sys
sys.path.append('..')

In [2]:
%matplotlib ipympl

In [3]:
from constellation import Constellation, Const
import numpy as np
import matplotlib.pyplot as plt

In [4]:
constellation = Constellation.createFromJson('../../constellationsTest.json', 'Starlink')

Загружена группировка Starlink


In [5]:
epochs = list(range(1002))

In [6]:
stateEci = constellation.propagateJ2(epochs)

In [7]:
np.shape(stateEci)

(2000, 3, 1002)

In [8]:
for ax in (0, 1, 2):
    print(f'ax / min / max = {ax} / {np.min(stateEci[:, ax, :])} / {np.max(stateEci[:, ax, :])}')

for ax in (0, 1, 2):
    print(f'ax / min / max = {ax} / {np.min(np.abs(stateEci[:, ax, :]))} / {np.max(np.abs(stateEci[:, ax, :]))}')

ax / min / max = 0 / -6878135.0 / 6878135.0
ax / min / max = 1 / -6878134.928942243 / 6878134.928942243
ax / min / max = 2 / -7071124.453339452 / 7071124.453339452
ax / min / max = 0 / 9.280903072166557e-10 / 6878135.0
ax / min / max = 1 / 0.0 / 6878134.928942243
ax / min / max = 2 / 0.0 / 7071124.453339452


In [9]:
stateEci[0, :, 0:10]

array([[ 6.87813500e+06,  6.87813415e+06,  6.87813161e+06,
         6.87812737e+06,  6.87812144e+06,  6.87811380e+06,
         6.87810448e+06,  6.87809346e+06,  6.87808074e+06,
         6.87806633e+06],
       [ 0.00000000e+00, -2.05930895e+03, -4.11861740e+03,
        -6.17792482e+03, -8.23723071e+03, -1.02965346e+04,
        -1.23558359e+04, -1.44151341e+04, -1.64744288e+04,
        -1.85337194e+04],
       [ 0.00000000e+00, -2.72430547e+03, -5.44861028e+03,
        -8.17291374e+03, -1.08972152e+04, -1.36215140e+04,
        -1.63458094e+04, -1.90701008e+04, -2.17943875e+04,
        -2.45186689e+04]])

#### Some visualizations of what is going on

In [10]:
def plotSphere(ax, r, **kwargs):
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = r * np.outer(np.cos(u), np.sin(v))
    y = r * np.outer(np.sin(u), np.sin(v))
    z = r * np.outer(np.ones(np.size(u)), np.cos(v))

    ax.plot_surface(x, y, z, **kwargs)

    return ax

def plotEarth(ax, **kwargs):
    nx, ny, nz = 0, 0, Const.earthRadius
    sx, sy, sz = 0, 0, -Const.earthRadius
    
    ax.scatter([nx], [ny], [nz], color='blue', s=2)
    ax.text(nx, ny, nz, 'N', color='blue')
    
    ax.scatter([sx], [sy], [sz], color='red', s=2)
    ax.text(sx, sy, sz, 'S', color='red')
    
    return plotSphere(ax, Const.earthRadius, **kwargs)

In [11]:
def plot_positions_at_epoch(stateEci, epochIdx):
    ax = plt.figure(figsize=(10, 10), num=f'satellites-epoch={epochIdx}').add_subplot(projection='3d')
    
    plotEarth(ax, alpha=0.25, color='gray')
    
    ax.scatter(
        stateEci[:, 0, epochIdx],
        stateEci[:, 1, epochIdx],
        stateEci[:, 2, epochIdx],
        s=1,
    )
    
    ax.set_title(f'positions of satellites at epoch={epochIdx}')
    ax.set_aspect('equal')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    return ax

In [12]:
# plot_positions_at_epoch(stateEci, 0)

In [13]:
def plot_trajectories(stateEci, satellites):
    ax = plt.figure(figsize=(10, 10), num='trajectories').add_subplot(projection='3d')
    
    plotEarth(ax, alpha=0.25, color='gray')
    
    for satIdx in satellites:
        ax.plot(
            stateEci[satIdx, 0, :],
            stateEci[satIdx, 1, :],
            stateEci[satIdx, 2, :],
        )
    
    ax.set_title(f'trajectories of {len(satellites)} satellites')
    ax.set_aspect('equal')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    return ax

In [14]:
# plot_trajectories(stateEci, range(0, stateEci.shape[0]))